# Interactively trigger GASpy tasks

- Wanted a notebook to interactively try GASpy calculations and tasks.

In [ ]:
import os

from ase.visualize import view
import tqdm
import matplotlib.pyplot as plt
import subprocess
import datetime

In [ ]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

## Routine tasks

- Here are some examples of tasks we'll routinely have to perform with GASpy.
- Some of these come from example scripts.

### Catalog population

In [ ]:
'''
This script will populate your `catalog` Mongo collection with adsorption sites
of alloys containing the given set of elements and with Miller indices no
higher than the specified `max_miller`.
'''

from gaspy.tasks.db_managers.catalog import update_catalog_collection


#elements = ['Ag', 'Al', 'As', 'Au', 'Bi', 'Ca', 'Cd', 'Cl', 'Co', 'Cr', 'Cs',
#            'Cu', 'Fe', 'Ga', 'Ge', 'H', 'Hf', 'Hg', 'In', 'Ir', 'K', 'Mn',
#            'Mo', 'N', 'Ni', 'Na', 'Nb', 'Os', 'P', 'Pb', 'Pd', 'Pt', 'Rb',
#            'Re', 'Rh', 'Ru', 'S', 'Sb', 'Sc', 'Se', 'Si', 'Sn', 'Sr', 'Ta',
#            'Tc', 'Te', 'Ti', 'Tl', 'V', 'W', 'Y', 'Zn', 'Zr']
max_miller = 2

elements = ['Pt', 'Ru', "Cu", "Ni", "Pd"]


update_catalog_collection(elements=elements, max_miller=max_miller, n_processes=1)

### Updating collections

In [ ]:
'''
This script will populate your `atoms` Mongo collection with completed
calculations in your FireWorks database.
'''
from gaspy.tasks.db_managers import update_all_collections


update_all_collections(n_processes=4)

### Triggering adsorption calculations

In [ ]:
all_site_documents = get_catalog_docs()

In [ ]:
from gaspy.tasks import schedule_tasks
from gaspy.gasdb import get_catalog_docs
from gaspy.tasks.metadata_calculators import CalculateAdsorptionEnergy


# Get all of the sites that we have enumerated
all_site_documents = get_catalog_docs()

# Pick the sites that we want to run. In this case, it'll be sites on
# palladium (as per Materials Project ID 2, mp-2) on (111) facets.
site_documents_to_calc = [doc for doc in all_site_documents
                          if (doc['mpid'] == 'mp-33' and
                              doc['miller'] == [1, 1, 1])]

# Turn the sites into GASpy/Luigi tasks
tasks = [CalculateAdsorptionEnergy(adsorbate_name='H',
                                   adsorption_site=doc['adsorption_site'],
                                   mpid=doc['mpid'],
                                   miller_indices=doc['miller'],
                                   shift=doc['shift'],
                                   top=doc['top'])
         for doc in site_documents_to_calc]

# Schedule/run all of the tasks
schedule_tasks(tasks)

## Introspection

- Contains recipes for inspecting things in the database and getting an idea of what's going on.

In [ ]:
from gaspy import fireworks_helper_scripts

### View atoms that match some Fireworks query

In [ ]:
lpad = fireworks_helper_scripts.get_launchpad()

In [ ]:
completed_fwids = lpad.get_fw_ids(query={"state": "COMPLETED", "name.adsorbate": ""})

In [ ]:
completed_atoms = [fireworks_helper_scripts.get_atoms_from_fwid(fwid=fwid) for fwid in tqdm.tqdm(completed_fwids)]

In [ ]:
view(completed_atoms)

### Check status of Fireworks jobs

In [ ]:
job_status = fireworks_helper_scripts.check_jobs_status(user_ID="samueldy", num_jobs=50)

In [ ]:
launch_dirs = job_status[job_status["state"]=="RUNNING"]

In [ ]:
launch_dirs

In [ ]:
for run in launch_dirs:
    print(run["fwid"])
    !ls -lah {run["launch_dir"]}

In [ ]:
job_status

In [ ]:
job_status.pivot_table(values="fwid", index="adsorbate", columns="shift", aggfunc="count")

### Inspect adsorption energies

In [ ]:
import gaspy.gasdb as gasdb
import seaborn as sns

In [ ]:
ads_docs = gasdb.get_adsorption_docs()

In [ ]:
energies = [doc["energy"] for doc in ads_docs]

In [ ]:
import pandas as pd

In [ ]:
energy_df = pd.DataFrame(data=ads_docs)

In [ ]:
# Swarm plot
sns.swarmplot(data=energy_df, y="energy", x="adsorbate")

In [ ]:
# Violin plot
fig, ax = plt.subplots(1, 1, figsize=(15, 8))
sns.violinplot(data=energy_df, y="energy", x="adsorbate", width=1, scale="count", ax=ax)
ax.set_xticks([0, 0.5, 1, 1.5])

## Maintenance

- This section has tasks for maintaining the database and other parts of the system.

### Back up Mongo database

- We should probably make this a cron job, but here's the template the Ulissigroup sent us.

In [ ]:
import pymongo

In [ ]:
from gaspy.utils import read_rc

In [ ]:
# Read variables
mongo_host = read_rc("fireworks_info.lpad.host")
mongo_port = read_rc("fireworks_info.lpad.port")
username = read_rc("fireworks_info.lpad.username")
password = read_rc("fireworks_info.lpad.password")

backup_location = read_rc("fireworks_info.backup_directory")

now = datetime.date.today().strftime("%Y_%m_%d")

In [ ]:
command = (
    f"""module load mongodb; mongodump --host "{mongo_host}" --port "{mongo_port}" """
    f"""--username "{username}" --password "{password}" """
    f"""--out "{os.path.join(backup_location, "mongodb_backup_" + now)}" """
    f"""--db "goldsmith-gaspy" --gzip"""
)

In [ ]:
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding="utf8", shell=True)

In [ ]:
print(result.stderr)

In [ ]:
k